# Notebook 01 — Dataset Construction from MIMIC-IV / MIMIC-IV-Note

Mục tiêu: xây dựng một tập dữ liệu gọn để huấn luyện mô hình văn bản → đa nhãn (ICD-block và nhóm xét nghiệm giai đoạn sớm), với khả năng giới hạn số dòng đọc ở mọi bước nhằm phục vụ demo nhanh và tiết kiệm tài nguyên.

Quy ước trình bày cho mỗi bước:
1) Mục đích của bước và lý do cần thiết.
2) Trường dữ liệu được sử dụng và ý nghĩa của từng trường; nếu nối giữa các bảng, chỉ rõ khóa nối và ý nghĩa của khóa.
3) Thực hiện xử lý dữ liệu.
4) Diễn giải ý nghĩa các trường trong kết quả hiển thị.

## 0) Cấu hình và nguyên tắc đọc dữ liệu có giới hạn

Bổ sung biến `SAMPLE_NROWS`. Khi đặt giá trị số dương, mọi lệnh đọc CSV đều thêm `nrows=SAMPLE_NROWS`, giúp demo nhanh mà không cần quét toàn bộ file. Nếu muốn đọc toàn bộ, đặt `SAMPLE_NROWS = None`.

Lưu ý: dùng `nrows` có thể khiến các khóa nối không phủ nhau hoàn toàn giữa các bảng. Để giữ tính nhất quán, sau khi xác định được tập `hadm_id` từ bước ICD, các bước sau sẽ lọc theo tập này trước khi nối thời gian hoặc gộp nhóm.

In [ ]:
# Cấu hình
from pathlib import Path

# Số dòng tối đa đọc từ mỗi CSV. None = đọc toàn bộ
SAMPLE_NROWS = 5000000

# Số lượng xét nghiệm sớm dùng làm nhãn (vocab) theo tần suất cao nhất
TOP_LABS = 50  # bạn có thể đổi 10, 20, 100, ...


# Tự dò DATA_ROOT
CANDIDATES = [Path("data"), Path("../data"), Path("../../data")] 
DATA_ROOT = None
for cand in CANDIDATES:
    if (cand / "mimiciv").exists() and (cand / "mimic-iv-note").exists():
        DATA_ROOT = cand
        break
if DATA_ROOT is None:
    raise FileNotFoundError("Không tìm thấy thư mục 'data' chứa 'mimiciv' và 'mimic-iv-note'. Hãy điều chỉnh CANDIDATES hoặc thiết lập DATA_ROOT thủ công.")

HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
ICU_DIR  = DATA_ROOT / "mimiciv" / "3.1" / "icu"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
PROC_DIR = DATA_ROOT / "proc"; PROC_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_ROOT:", DATA_ROOT.resolve())
print("HOSP_DIR:", HOSP_DIR.resolve())
print("NOTE_DIR:", NOTE_DIR.resolve())
print("PROC_DIR:", PROC_DIR.resolve())

DATA_ROOT: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data
HOSP_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/hosp
NOTE_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimic-iv-note/2.2/note
PROC_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc


Tiện ích đọc dữ liệu có giới hạn số dòng và vài hàm xử lý cơ bản:
- `read_csv_limited`: bọc `pandas.read_csv` để tự động gắn `nrows=SAMPLE_NROWS` khi cần.
- `normalize_text`: chuẩn hóa văn bản ghi chú.
- `icd_to_block`: rút gọn mã ICD về ICD-block (3 ký tự đầu, bỏ dấu chấm).

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter

def read_csv_limited(path, **kwargs):
    """Đọc CSV với compression='gzip' mặc định và áp dụng nrows=SAMPLE_NROWS nếu được cấu hình."""
    kwargs.setdefault("compression", "gzip")
    if SAMPLE_NROWS is not None:
        kwargs.setdefault("nrows", SAMPLE_NROWS)
    return pd.read_csv(path, **kwargs)

def normalize_text(s):
    if pd.isna(s):
        return ""
    return " ".join(str(s).split())[:4000]

def icd_to_block(icd_code: str) -> str:
    if pd.isna(icd_code):
        return np.nan
    s = str(icd_code).replace('.', '').strip()
    return s[:3] if len(s) >= 3 else s

pd.set_option("display.max_colwidth", 180)
pd.set_option("display.width", 140)

## 1) Admissions: mốc thời gian nhập viện

1) Mục đích: lấy mốc `admittime` cho từng `hadm_id` để xác định cửa sổ thời gian sớm cho xét nghiệm (0–6 giờ) và ghi chú (0–12 giờ).
2) Trường dùng: `hadm_id` (khóa lần nhập viện), `admittime` (thời điểm nhập viện). `hadm_id` là khóa để nối với các bảng khác như xét nghiệm hoặc ghi chú.
3) Xử lý: đọc file admissions, chuẩn hóa thời gian, đặt index theo `hadm_id`.
4) Kết quả: bảng tra cứu `admittime` theo `hadm_id`.


In [3]:
adm_path = HOSP_DIR / "admissions.csv.gz"
admissions = read_csv_limited(adm_path, usecols=["hadm_id", "admittime"], parse_dates=["admittime"])
admissions = admissions.dropna(subset=["admittime"]).drop_duplicates("hadm_id").set_index("hadm_id")
display(admissions.head())

print("Giải thích kết quả:")
print("- hadm_id: mã lần nhập viện.")
print("- admittime: thời điểm nhập viện, dùng làm mốc để tính 0–6h và 0–12h.")

,admittime
hadm_id,
22595853,2180-05-06 22:23:00
22841357,2180-06-26 18:27:00
25742920,2180-08-05 23:44:00
29079034,2180-07-23 12:35:00
25022803,2160-03-03 23:16:00


Giải thích kết quả:
- hadm_id: mã lần nhập viện.
- admittime: thời điểm nhập viện, dùng làm mốc để tính 0–6h và 0–12h.


## 2) ICD-block cho từng lần nhập viện

1) Mục đích: rút gọn các mã ICD chi tiết thành ICD-block để giảm số nhãn và giữ ý nghĩa nhóm bệnh chính.
2) Trường dùng: `subject_id` (mã bệnh nhân), `hadm_id` (mã lần nhập viện), `icd_code` (mã bệnh). Khóa nối khi cần về sau là `hadm_id`.
3) Xử lý: đọc `diagnoses_icd.csv.gz` (giới hạn số dòng), chuyển mỗi `icd_code` về ICD-block (3 ký tự đầu). Gom các block theo `hadm_id`.
4) Kết quả: khung dữ liệu gồm `hadm_id`, `subject_id`, `icd_blocks` (danh sách ICD-block của ca đó).

In [4]:
diag_path = HOSP_DIR / "diagnoses_icd.csv.gz"
diag_chunk = read_csv_limited(diag_path, usecols=["subject_id", "hadm_id", "icd_code"])  # giới hạn bằng SAMPLE_NROWS
diag_chunk["block"] = diag_chunk["icd_code"].map(icd_to_block)

icd_df = (diag_chunk
          .dropna(subset=["hadm_id"]) 
          .groupby(["subject_id", "hadm_id"])  
          ["block"].apply(lambda x: sorted(set([b for b in x if pd.notna(b)])))
          .reset_index()
          .rename(columns={"block": "icd_blocks"}))

display(icd_df.head())

print("Giải thích kết quả:")
print("- subject_id: mã bệnh nhân.")
print("- hadm_id: mã lần nhập viện (khóa nối với xét nghiệm và ghi chú).")
print("- icd_blocks: danh sách ICD-block (3 ký tự đầu) đại diện nhóm bệnh chính của ca.")

# Tập hadm_id dùng cho các bước tiếp theo để đảm bảo nhất quán khi chỉ đọc một phần dữ liệu
keep_hadm = set(icd_df["hadm_id"].unique())
len_keep_hadm = len(keep_hadm)
print("Số hadm_id trong tập demo:", len_keep_hadm)

# ---- BƯỚC 2.5: Thêm demographics (gender) + tuổi tại thời điểm nhập viện ----
pat_path = HOSP_DIR / "patients.csv.gz"

# Đọc demographics từ patients
patients = read_csv_limited(
    pat_path,
    usecols=["subject_id", "gender", "anchor_age", "anchor_year"]
)
patients["gender"] = patients["gender"].astype(str).str.upper().str[0]  # 'M'/'F'/'U'

# Map hadm_id -> subject_id từ icd_df
hadm_subject = icd_df[["hadm_id", "subject_id"]].drop_duplicates()

# Lấy năm nhập viện từ admissions (đã có ở BƯỚC 1)
adm_year = admissions.reset_index()[["hadm_id", "admittime"]].copy()
adm_year["admit_year"] = adm_year["admittime"].dt.year

# Ghép để tính tuổi tại nhập viện: age_at_admit = anchor_age + (admit_year - anchor_year)
age_df = (
    hadm_subject
    .merge(patients, on="subject_id", how="left")
    .merge(adm_year[["hadm_id", "admit_year"]], on="hadm_id", how="left")
)
age_df["age_at_admit"] = age_df["anchor_age"] + (age_df["admit_year"] - age_df["anchor_year"])

# Làm sạch tuổi (0..120), làm tròn, giữ kiểu số nguyên nullable
age_df["age_at_admit"] = (
    age_df["age_at_admit"]
    .clip(lower=0, upper=120)
    .round()
    .astype("Int64")
)

# Chỉ giữ cột cần thiết để merge vào df cuối
demo_df = age_df[["hadm_id", "subject_id", "gender", "age_at_admit"]].copy()


,subject_id,hadm_id,icd_blocks
0,10000032,22595853,"[070, 296, 309, 496, 571, 572, 789, V15]"
1,10000032,22841357,"[070, 276, 287, 305, 496, 571, 789, V08]"
2,10000032,25742920,"[070, 276, 305, 496, 571, 787, 789, V08, V46]"
3,10000032,29079034,"[070, 276, 296, 305, 458, 496, 571, 789, 799, V08, V46, V49]"
4,10000068,25022803,[305]


Giải thích kết quả:
- subject_id: mã bệnh nhân.
- hadm_id: mã lần nhập viện (khóa nối với xét nghiệm và ghi chú).
- icd_blocks: danh sách ICD-block (3 ký tự đầu) đại diện nhóm bệnh chính của ca.
Số hadm_id trong tập demo: 85520


## 3) Nhóm xét nghiệm trong 6 giờ đầu

1) Mục đích: xây dựng nhãn về các nhóm xét nghiệm được thực hiện sớm, nhằm huấn luyện mô hình gợi ý cận lâm sàng.
2) Trường dùng và khóa nối:
- Từ `labevents.csv.gz`: `hadm_id` (khóa lần nhập viện), `itemid` (mã xét nghiệm), `charttime` (thời điểm xét nghiệm).
- Từ `d_labitems.csv.gz`: `itemid` (khóa), `label` (tên xét nghiệm, dùng để suy ra nhóm).
- Từ `admissions` (đã đọc ở Bước 1): `hadm_id` (khóa), `admittime` (mốc thời gian).
Nối theo `hadm_id` giữa `labevents` và `admissions` để tính thời gian tương đối; ánh xạ `itemid` sang nhóm xét nghiệm qua `d_labitems`.
3) Xử lý: chỉ đọc số dòng theo `SAMPLE_NROWS`, lọc `hadm_id` thuộc tập demo, tính khoảng cách thời gian và giữ các xét nghiệm trong 0–6 giờ đầu; gán nhóm xét nghiệm theo từ khóa đơn giản.
4) Kết quả: khung dữ liệu `labs_df` gồm `hadm_id` và `lab_groups` (danh sách nhóm xét nghiệm trong 6 giờ đầu).

In [5]:
# ---- BƯỚC 3: Lấy xét nghiệm sớm Top-N theo tần suất (0–6h) ----
labs_path = HOSP_DIR / "labevents.csv.gz"
dlab_path = HOSP_DIR / "d_labitems.csv.gz"

# 3.1 Đọc từ điển lab: itemid -> label
dlab = read_csv_limited(dlab_path, usecols=["itemid", "label"])
dlab["itemid"] = dlab["itemid"].astype(int)
dlab["label"] = dlab["label"].astype(str)

# 3.2 Đọc labs (giới hạn theo SAMPLE_NROWS), lọc theo hadm demo
labs_raw = read_csv_limited(labs_path, usecols=["hadm_id", "itemid", "charttime"])
labs_raw = labs_raw.dropna(subset=["hadm_id", "itemid", "charttime"]).copy()
labs_raw["hadm_id"] = labs_raw["hadm_id"].astype(int)
labs_raw["itemid"] = labs_raw["itemid"].astype(int)
labs_raw = labs_raw[labs_raw["hadm_id"].isin(keep_hadm)]

# 3.3 Nối với admissions để tính khoảng thời gian
if not labs_raw.empty:
    labs_raw["charttime"] = pd.to_datetime(labs_raw["charttime"], errors="coerce")
    merged = labs_raw.join(admissions, on="hadm_id", how="inner")
    dt_hours = (merged["charttime"] - merged["admittime"]).dt.total_seconds() / 3600.0
    early = merged[(dt_hours >= 0) & (dt_hours <= 6)].copy()
else:
    early = pd.DataFrame(columns=["hadm_id","itemid","charttime"])

# 3.4 Chọn TOP_N itemid theo tần suất
if not early.empty:
    counts = (early["itemid"]
              .value_counts()
              .rename_axis("itemid")
              .reset_index(name="count"))
    # Thứ tự ổn định: đếm giảm dần, rồi theo itemid tăng dần (ràng buộc tie-break)
    counts = counts.sort_values(["count","itemid"], ascending=[False, True])
    top_items = counts.head(TOP_LABS)["itemid"].tolist()

    # Tạo vocab xét nghiệm: itemid + label + count
    lab_vocab_df = (counts[counts["itemid"].isin(top_items)]
                    .merge(dlab, on="itemid", how="left"))
    lab_vocab_df = lab_vocab_df[["itemid","label","count"]].reset_index(drop=True)

    # 3.5 Tạo nhãn theo hadm_id: chỉ giữ itemid thuộc vocab
    early_top = early[early["itemid"].isin(top_items)].copy()
    lab_items_by_hadm = (early_top.groupby("hadm_id")["itemid"]
                         .apply(lambda s: sorted(set(s.tolist())))
                         .reset_index()
                         .rename(columns={"itemid": "lab_items"}))
else:
    lab_vocab_df = pd.DataFrame(columns=["itemid","label","count"])
    lab_items_by_hadm = pd.DataFrame(columns=["hadm_id","lab_items"])

# Kết quả bước 3
display(lab_vocab_df.head(10))
display(lab_items_by_hadm.head(10))
print("Số phần tử vocab xét nghiệm (TOP_N):", len(lab_vocab_df))
print("Số ca có ít nhất một xét nghiệm trong TOP_N:", len(lab_items_by_hadm))


,itemid,label,count
0,51221,Hematocrit,793
1,51265,Platelet Count,769
2,51222,Hemoglobin,746
3,51248,MCH,745
4,51249,MCHC,745
5,51250,MCV,745
6,51277,RDW,745
7,51279,Red Blood Cells,745
8,51301,White Blood Cells,745
9,50971,Potassium,733


,hadm_id,lab_items
0,20010003,"[50861, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50911, 50912, 50920, 50931, 50960, 50970, 50971, 50983, 51003, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, ..."
1,20016088,"[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301, 51678, 52172]"
2,20023045,[51003]
3,20025657,"[50813, 50912, 50920, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301]"
4,20044587,"[50802, 50804, 50813, 50818, 50820, 50821, 50868, 50882, 50902, 50912, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301, ..."
5,20051301,"[50868, 50882, 50893, 50902, 50912, 50920, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, ..."
6,20061526,"[51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301]"
7,20090856,"[50813, 50820, 50868, 50882, 50893, 50902, 50911, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51003, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, ..."
8,20093566,"[50802, 50804, 50813, 50818, 50820, 50821, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301, 52033, 52172]"
9,20097154,"[51464, 51466, 51478, 51484, 51486, 51487, 51491, 51492, 51498]"


Số phần tử vocab xét nghiệm (TOP_N): 50
Số ca có ít nhất một xét nghiệm trong TOP_N: 1121


## 4) Văn bản ghi chú Radiology trong 12 giờ đầu

1) Mục đích: trích một đoạn văn bản sớm làm đầu vào cho mô hình văn bản → nhãn; ví dụ chọn radiology note sớm nếu có.
2) Trường dùng và khóa nối:
- Từ `radiology.csv.gz`: `hadm_id` (khóa), `charttime` (thời điểm ghi chú), `text` (nội dung ghi chú).
- Từ `admissions`: `hadm_id` (khóa), `admittime` (mốc thời gian).
Nối theo `hadm_id` để tính chênh lệch thời gian và lọc ghi chú trong 0–12 giờ đầu.
3) Xử lý: chỉ đọc số dòng theo `SAMPLE_NROWS`, lọc `hadm_id` thuộc tập demo, chuẩn hóa `text`, chọn ghi chú sớm nhất cho mỗi `hadm_id`.
4) Kết quả: khung dữ liệu `text_df` gồm `hadm_id` và `text` đại diện sớm (nếu có).

In [6]:
rad_path = NOTE_DIR / "radiology.csv.gz"
rad_raw = read_csv_limited(rad_path, usecols=["hadm_id", "charttime", "text"])  # giới hạn bằng SAMPLE_NROWS
rad_raw = rad_raw.dropna(subset=["hadm_id", "charttime", "text"]).copy()
rad_raw["hadm_id"] = rad_raw["hadm_id"].astype(int)
rad_raw = rad_raw[rad_raw["hadm_id"].isin(keep_hadm)]  # lọc theo tập demo từ bước ICD

if not rad_raw.empty:
    rad_raw["charttime"] = pd.to_datetime(rad_raw["charttime"], errors="coerce")
    rad_raw["text"] = rad_raw["text"].map(normalize_text)
    merged = rad_raw.join(admissions, on="hadm_id", how="inner")
    dt_hours = (merged["charttime"] - merged["admittime"]).dt.total_seconds() / 3600.0
    early = merged[(dt_hours >= 0) & (dt_hours <= 12)].copy()
    early = early.sort_values(["hadm_id", "charttime"])  # chọn sớm nhất
    first_text = early.groupby("hadm_id")["text"].first().reset_index()
    text_df = first_text
else:
    text_df = pd.DataFrame(columns=["hadm_id", "text"])

display(text_df.head())

print("Giải thích kết quả:")
print("- hadm_id: mã lần nhập viện, trùng với các bảng trước.")
print("- text: văn bản ghi chú radiology sớm nhất trong 12 giờ đầu, sau chuẩn hóa cơ bản.")

,hadm_id,text
0,20000057,"CLINICAL HISTORY: Fell on buttocks, chronic left hip osteoarthritis. Evaluate for fracture. PELVIS AND LEFT HIP Total hip replacement is present on the right side. This appears..."
1,20000254,INDICATION: Evaluate for obstruction in a ___ woman with abdominal pain and constipation. TECHNIQUE: Frontal supine and upright abdominal radiographs were obtained. COMPARISON:...
2,20001297,INDICATION: History: ___ with weakness // ? pna TECHNIQUE: AP and lateral images of the chest. COMPARISON: None. FINDINGS: The lungs are hyperinflated but clear. There is no pl...
3,20001811,INDICATION: ___ year old man with cough// c/f pneumonia or other acute pulmonary process TECHNIQUE: Frontal radiograph of the chest. COMPARISON: ___ 08:54. IMPRESSION: Right-si...
4,20001956,"INDICATION: ___ man with end-stage liver disease and TIPS, evaluate for TIPS occlusion. COMPARISON: Liver Doppler ultrasound on ___. FINDINGS: The liver demonstrates a coarsene..."


Giải thích kết quả:
- hadm_id: mã lần nhập viện, trùng với các bảng trước.
- text: văn bản ghi chú radiology sớm nhất trong 12 giờ đầu, sau chuẩn hóa cơ bản.


## 5) Ghép dữ liệu và sinh nhãn đa nhãn (multi-hot)

1) Mục đích: ghép các phần thông tin (ICD-block, nhóm xét nghiệm, văn bản) thành một bảng duy nhất cho huấn luyện.
2) Khóa nối: `hadm_id` giữa các bảng `icd_df`, `labs_df`, `text_df`.
3) Xử lý: nối dữ liệu, tạo từ điển nhãn (vocabulary) và sinh vector multi-hot cho ICD-block và nhóm xét nghiệm.
4) Kết quả: bảng cuối cùng gồm `hadm_id`, `subject_id`, `text`, `icd_blocks`, `lab_groups`, `y_icd`, `y_lab`. Đồng thời lưu `examples.parquet` và `vocab_meta.json` để dùng trong bước huấn luyện mô hình.

In [7]:
# ---- BƯỚC 5 (cập nhật): Merge và multi-hot theo vocab Top-N ----

# 5.1 Ghép theo hadm_id + subject_id (thêm demographics)
df = (
    icd_df
    .merge(lab_items_by_hadm, on="hadm_id", how="left")
    .merge(text_df, on="hadm_id", how="left")
    .merge(demo_df, on=["hadm_id", "subject_id"], how="left")  # <<-- thêm tuổi + giới tính
)

# Chuẩn hóa list rỗng, text và demographics
df["icd_blocks"] = df["icd_blocks"].apply(lambda x: x if isinstance(x, list) else [])
df["lab_items"]  = df["lab_items"].apply(lambda x: x if isinstance(x, list) else [])
df["text"] = df["text"].fillna("")
df["gender"] = df["gender"].fillna("U")                  # Unknown nếu thiếu
df["age_at_admit"] = df["age_at_admit"].astype("Int64")  # giữ nullable int

# 5.2 Vocabulary
# ICD-block: Top-2000 (giữ nguyên như trước)
from collections import Counter
cnt_icd = Counter(b for blocks in df["icd_blocks"] for b in blocks)
icd_vocab = [b for b, _ in cnt_icd.most_common(100)]
icd_index = {b: i for i, b in enumerate(icd_vocab)}

# Lab items: từ lab_vocab_df (Top-N theo tần suất)
lab_vocab_items = lab_vocab_df["itemid"].tolist()
lab_index = {it: i for i, it in enumerate(lab_vocab_items)}

# 5.3 Hàm multi-hot
def to_multihot_generic(labels, index_map, length):
    arr = np.zeros(length, dtype=np.int8)
    for t in labels:
        if t in index_map:
            arr[index_map[t]] = 1
    return arr

df["y_icd"] = df["icd_blocks"].apply(lambda xs: to_multihot_generic(xs, icd_index, len(icd_vocab)))
df["y_lab"] = df["lab_items"].apply(lambda xs: to_multihot_generic(xs, lab_index, len(lab_vocab_items)))

# 5.4 Ánh xạ itemid -> label để tiện hiển thị
itemid_to_label = dict(zip(lab_vocab_df["itemid"], lab_vocab_df["label"]))

display(df.head())
print("Kích thước vocab ICD:", len(icd_vocab))
print("Kích thước vocab Lab (Top-N):", len(lab_vocab_items))


,subject_id,hadm_id,icd_blocks,lab_items,text,gender,age_at_admit,y_icd,y_lab
0,10000032,22595853,"[070, 296, 309, 496, 571, 572, 789, V15]","[51464, 51466, 51478, 51484, 51486, 51487, 51491, 51492, 51498]","EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year-old female with cirrhosis, jaundice. TECHNIQUE: Grey scale and color Doppler ultrasound images of the a...",F,52,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,10000032,22841357,"[070, 276, 287, 305, 496, 571, 789, V08]","[51464, 51466, 51478, 51484, 51486, 51487, 51491, 51492, 51498]",,F,52,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,10000032,25742920,"[070, 276, 305, 496, 571, 787, 789, V08, V46]",[],"EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ORGAN) INDICATION: ___ year old woman with cirrhosis, p/w abd pain as well as elevated alk phos // eval for biliary pathology and P...",F,52,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000032,29079034,"[070, 276, 296, 305, 458, 496, 571, 789, 799, V08, V46, V49]",[],,F,52,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000068,25022803,[305],[],,F,19,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


Kích thước vocab ICD: 100
Kích thước vocab Lab (Top-N): 50


## 6) Kiểm tra nhanh

1) Mục đích: xác nhận dữ liệu đã ghép hợp lệ và có đủ thông tin tối thiểu cho huấn luyện demo.
2) Trường dùng: thống kê đơn giản trên các cột đầu ra.
3) Xử lý: tính tỉ lệ có văn bản, trung bình số nhãn.
4) Kết quả: số liệu tổng quan để quyết định có cần điều chỉnh `SAMPLE_NROWS` hay logic lọc hay không.

In [8]:
# ---- XUẤT (cập nhật) ----
out_examples = PROC_DIR / "examples.parquet"

df_out = df[[
    "hadm_id",
    "subject_id",
    "gender",        # <<-- thêm
    "age_at_admit",  # <<-- thêm
    "text",
    "icd_blocks",
    "lab_items",
    "y_icd",
    "y_lab"
]].copy()

df_out.to_parquet(out_examples, index=False)

out_vocab = PROC_DIR / "vocab_meta.json"
import json
with open(out_vocab, "w") as f:
    json.dump({
        "icd_vocab": icd_vocab,
        "n_icd": len(icd_vocab),
        "lab_vocab_items": lab_vocab_items,
        "n_lab": len(lab_vocab_items),
        "itemid_to_label": {int(k): str(v) for k, v in itemid_to_label.items()}
    }, f, indent=2)

print("Đã lưu:")
print("-", out_examples)
print("-", out_vocab)


Đã lưu:
- ../data/proc/examples.parquet
- ../data/proc/vocab_meta.json
